Before running the notebook follow these steps:
- Export the labelled images from label studio and save them in a folder called labelled_images.
- Seperate into images and labels folders, containing train, test and validation.
- Ensure the split is made with enough background images in the all subsets.
- Have the data and hyp yaml files in the direectory you are running this notebook
- Update the data.yaml file to have the correct paths


Clone the Yolov5 repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Install the requirements of the project

In [ ]:
!pip install -r yolov5/requirements.txt


Download the pre-trained weights for the yolov5 model and run an inital detection on the test set

In [ ]:
!python yolov5/detect.py --weights yolov5s.pt --source labelled_images/images/test


Fine tune model by training & validating on our labelled images


In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 10 --data data.yaml --weights yolov5/yolov5s.pt --hyp hyp.yaml --cache

Results of the runs can be seen in runs/train/exp directory.

Then test the model on the test images to see how well it label them.

In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp7/weights/best.pt --img 640 --conf 0.50 --source labelled_images/images/test
#update the folder number wiith the best weights


Then run best model on all the remaining images to label them

In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp7/weights/best.pt --img 640 --conf 0.50 --source all_images


Then install ONNX, onnx-simplifier, onnx-tf and TensorFlow as we want to save the trained model and it's weights to a TensorFlow compatible format

In [ ]:
!pip install torch onnx onnx-simplifier onnx-tf tensorflow


Then export the fine tunned YOLOv5 model (best.pt) to ONNX format

In [ ]:
!python yolov5/export.py --weights yolov5/runs/train/exp7/weights/best.pt  --img-size 640 --batch 1 --device cpu --simplify --export onnx


Then simplify the ONNX model to make it more compatible with tensorflow

In [ ]:
!python -m onnxsim yolov5/yolov5s.onnx yolov5/yolov5s_simplified.onnx


Finally convert ONNX model to  the TensorFlow format

In [ ]:
!onnx-tf convert -i yolov5/yolov5s_simplified.onnx -o yolov5/yolov5_tf


Save the yolov5_tf file to be able to load it for furture use